# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125562e+02     1.549688e+00
 * time: 0.12113690376281738
     1     1.097746e+01     8.160994e-01
 * time: 1.3151359558105469
     2    -1.229744e+01     9.484659e-01
 * time: 1.4491558074951172
     3    -3.414772e+01     7.162399e-01
 * time: 1.617602825164795
     4    -4.766143e+01     5.200478e-01
 * time: 1.7685678005218506
     5    -5.687032e+01     2.115546e-01
 * time: 1.9128808975219727
     6    -5.971531e+01     1.544505e-01
 * time: 2.0192699432373047
     7    -6.083312e+01     5.471658e-02
 * time: 2.132413864135742
     8    -6.128791e+01     5.505455e-02
 * time: 2.2450649738311768
     9    -6.154961e+01     5.273947e-02
 * time: 2.351799964904785
    10    -6.176752e+01     2.728432e-02
 * time: 2.4665069580078125
    11    -6.193440e+01     2.725226e-02
 * time: 2.5799148082733154
    12    -6.203131e+01     2.200378e-02
 * time: 2.6878907680511475
    13    -6.210494e+01     1.622558e-02
 * time: 2.802043914794

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671058
    AtomicLocal         -18.8557632
    AtomicNonlocal      14.8522620
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485354 
    Xc                  -19.3336813

    total               -62.261666461686
